# Visualize a scattering event using my WebGL code

You can use the [live version](https://ageller.github.io/Scatter_WebGL/) to drag-and-drop your own file and load your data into the viewer.  But if you want to run this locally (e.g., so that you can avoid the drag-and-drop feature and load your file by default), here are the steps you can follow. 

First, clone the GitHub repo: https://github.com/ageller/Scatter_WebGL

This code will ingest data in .csv format that has columns of :

```ID,time,x,y,z```

Each particle gets it's own row during each time step (so you will have repeated values of time).  An example csv file is provided in the ```data``` directory.  You will need to copy your data to that ```data``` directory and name the file ```ScatterParts.csv```.  This will then become the default dataset that loads when you launch the viewer.

Below is Python code that can spawn a server and show the viewer in a notebook.  

Alternatively, you can start a server from your own terminal (without the need for this notebook) with :

```python -m http.server```

Then you can point your browser to the url ```http://localhost:8000```.

In [ ]:
import os

from IPython.display import IFrame

import http.server
import socketserver

from threading import Thread

In [ ]:
# define the directory where the code lives (this can be anywhere on your computer; just change the directory below)
directory = os.path.join(os.getcwd())

In [ ]:
# define the port that you want (8000 is standard)
port = 8001

In [ ]:
# a simple class to start the http server in a thread (so that you still have access to the jupyter notebook)
class serverThread(Thread): 
    def run(self):
        handler = http.server.SimpleHTTPRequestHandler
        os.chdir(directory)
        with socketserver.TCPServer(("", port), handler) as httpd:
            print("serving at port", port)
            httpd.serve_forever()

serverThread().start()

In [ ]:
# create an iFrame to view the visualization in this notebook
IFrame("http://localhost:" + str(port), width = 800, height = 500)